In [1]:
%%writefile requirements.txt
Flask==0.12.2
flask-socketio
eventlet==0.17.4
gunicorn==18.0.0

Overwriting requirements.txt


In [2]:

from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:

%%writefile main.py
import os
from flask import Flask, render_template, request, send_from_directory
from fastai.vision import *
from fastai import *
import matplotlib.pyplot as plt
#import torch
from flask_socketio import SocketIO

secret = os.urandom(24).hex()
app = Flask(__name__)
app.logger.info("Starting...")
app.config['SECRET_KEY'] = secret
app.logger.critical("secret: %s" % secret)
socketio = SocketIO(app)


# path to the pre-trained model
STATIC_FOLDER = '/content/gdrive/My Drive/Model/'


def load__model():
    global model
    model = load_learner(STATIC_FOLDER)


def predict(fullpath):
    data = open_image(fullpath)
    pred_class,pred_idx,outputs = model.predict(data)
    return pred_class


# Home Page
@app.route('/')
def index():
    return render_template('index.html')


# Process file and predict his label
@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        file = request.files['image']
        fullname = os.path.join(STATIC_FOLDER, file.filename)
        file.save(fullname)
        label = predict(fullname)
        return render_template('predict.html', image_file_name=file.filename, label=label)

@app.route('/about')
def about():
    return render_template('about.html')
@app.route('/upload/<filename>')
def send_file(filename):
    return send_from_directory(STATIC_FOLDER, filename)


def create_app():
    load__model()
    return app


if __name__ == '__main__':
    app = create_app()
    socketio.run(app, debug=True)

Overwriting main.py


In [4]:
!mkdir static

mkdir: cannot create directory ‘static’: File exists


In [5]:
from shutil import copyfile
src = "/content/gdrive/My Drive/Model/img.svg"
dst = "static/img.svg"
copyfile(src, dst)

'static/img.svg'

In [6]:
%%writefile static/style.css
@import url("https://fonts.googleapis.com/css2?family=Poppins:wght@400;600;700&display=swap");

:root{
  --header-height: 3rem;
  --font-semi: 600;
}

:root{
  --first-color: #f27d0c;
  --second-color: #0E2431;
}


:root{
  --body-font: 'Poppins', sans-serif;
  --big-font-size: 1.7rem;
  --h2-font-size: 1.25rem;
  --normal-font-size: 0.938rem;
}
@media screen and (min-width: 768px){
  :root{
    --big-font-size: 2.5rem;
    --h2-font-size: 2rem;
    --normal-font-size: 1rem;
  }
}

/* Margenes */
:root{
  --mb-1: 0.5rem;
  --mb-2: 1rem;
  --mb-3: 1.5rem;
  --mb-4: 2rem;
  --mb-5: 2.5rem;
  --mb-6: 3rem;
}

/* z index */
:root{
  --z-back: -10;
  --z-normal: 1;
  --z-tooltip: 10;
  --z-fixed: 100;
}

*,::before,::after{
  box-sizing: border-box;
}
html{
  scroll-behavior: smooth;
}
body{
  margin: var(--header-height) 0 0 0;
  font-family: var(--body-font);
  font-size: var(--normal-font-size);
  color: var(--second-color);
}
h1,h2,p{
  margin: 0;
}
ul{
  margin: 0;
  padding: 0;
  list-style: none;
}
a{
  text-decoration: none;
}
img{
  max-width: 100%;
  height: auto;
  display: block;
}

/*===== CLASS CSS ===== */
.section-title{
  position: relative;
  font-size: var(--h2-font-size);
  color: var(--first-color);
  margin-top: var(--mb-2);
  margin-bottom: var(--mb-4);
  text-align: center;
}
.section-title::after{
  position: absolute;
  content: "";
  width: 64px;
  height: 0.18rem;
  left: 0;
  right: 0;
  margin: auto;
  top: 2rem;
  background-color: var(--first-color);
}
.section{
  padding-top: 3rem;
  padding-bottom: 2rem;
}

.bd-grid{
  max-width: 1024px;
  display: grid;
  grid-template-columns: 100%;
  grid-column-gap: 2rem;
  width: calc(100% - 2rem);
  margin-left: var(--mb-2);
  margin-right: var(--mb-2);
}
.l-header{
  width: 100%;
  position: fixed;
  top: 0;
  left: 0;
  z-index: var(--z-fixed);
  background-color: #fff;
  box-shadow: 0 1px 4px rgba(146,161,176,.15);
}


.nav{
  height: var(--header-height);
  display: flex;
  justify-content: space-between;
  align-items: center;
  font-weight: var(--font-semi);
}
@media screen and (max-width: 768px){
  .nav__menu{
    position: fixed;
    top: var(--header-height);
    right: -100%;
    width: 80%;
    height: 100%;
    padding: 2rem;
    background-color: var(--second-color);
    transition: .5s;
  }
}
.nav__item{
  margin-bottom: var(--mb-4);
}
.nav__link{
  position: relative;
  color: #fff;
}
.nav__link:hover{
  position: relative;
}
.nav__link:hover::after{
  position: absolute;
  content: "";
  width: 100%;
  height: 0.18rem;
  left: 0;
  top: 2rem;
  background-color: var(--first-color);
}
.nav__logo{
  color: var(--second-color);
}
.nav__toggle{
  color: var(--second-color);
  font-size: 1.5rem;
  cursor: pointer;
}

.active::after{
  position: absolute;
  content: "";
  width: 100%;
  height: 0.18rem;
  left: 0;
  top: 2rem;
  background-color: var(--first-color);
}

.show{
  right: 0;
}

.home{
  height: calc(100vh - 3rem);
  row-gap: 1rem;
}
.home__data{
  align-self: center;
}
.home__title{
  font-size: var(--big-font-size);
  margin-bottom: var(--mb-5);
}
.c__title{
	font-size: 1.2 rem;
	margin-bottom: var(--mb-5);
}
.home__title-color{
  color: var(--first-color);
}

.home__social-icon{
  width: max-content;
  margin-bottom: var(--mb-2);
  font-size: 1.5rem;
  color: var(--second-color);
}
.home__social-icon:hover{
  color: var(--first-color);
}
.home__img{
	
  position: absolute;
  right: 0;
  bottom: 0;
  width: 250px;
}

.button{
  display: inline-block;
  background-color: var(--first-color);
  color: #fff;
  padding: .75rem 2.5rem;
  font-weight: var(--font-semi);
  border-radius: .5rem;
  border: 1px solid #fff;
}
.button:hover{
  box-shadow: 0 10px 36px rgba(0,0,0,.15);
}

input[type="file"] {
    display: none;
}
input[type="submit"] {
    display: none;
}
.custom-file-upload {
    border: 1px solid var(--first-color);
    display: inline-block;
    padding: 6px 12px;
    cursor: pointer;
}
.custom-file-upload:hover{ 
	box-shadow: 0 10px 36px rgba(0,0,0,.15);
	background-color: var(--first-color);
	color: #fff;
	font-weight: var(--font-semi);
}


.footer{
  background-color: #f5a454;
  color: #fff;
  text-align: center;
  font-weight: var(--font-semi);
  padding: 2rem 0;
}
.footer__title{
  font-size: 2rem;
  margin-bottom: var(--mb-4);
}

@media screen and (min-width: 768px){
  body{
    margin: 0;
  }
  .section{
    padding-top: 4rem;
    padding-bottom: 3rem;
  }
  .section-title{
    margin-bottom: var(--mb-6);
  }
  .section-title::after{
    width: 80px;
    top: 3rem;
  }

  .nav{
    height: calc(var(--header-height) + 1rem);
  }
  .nav__list{
    display: flex;
    padding-top: 0;
  }
  .nav__item{
    margin-left: var(--mb-6);
    margin-bottom: 0;
  }
  .nav__toggle{
    display: none;
  }
  .nav__link{
    color: var(--second-color);
  }

  .home{
    height: 100vh;
  }
  .home__data{
    align-self: flex-end;
  }
  .home__social-icon{
    margin-bottom: 0;
    margin-right: var(--mb-4);
  }
  .home__img{
    width: 400px;
    bottom: 15%;
  }

  .about__container, .skills__container{
    grid-template-columns: repeat(2,1fr);
    align-items: center;
    text-align: initial;
  }
  .about__img img{
    width: 200px;
  }
}

@media screen and (min-width: 1024px){
  .bd-grid{
    margin-left: auto;
    margin-right: auto;
  }
  .home__img{
    right: 10%;
  }
}





Overwriting static/style.css


In [7]:
%%writefile static/main.js

const showMenu = (toggleId, navId) =>{
    const toggle = document.getElementById(toggleId),
    nav = document.getElementById(navId)

    if(toggle && nav){
        toggle.addEventListener('click', ()=>{
            nav.classList.toggle('show')
        })
    }
}
showMenu('nav-toggle','nav-menu')
const navLink = document.querySelectorAll('.nav__link');   

function linkAction(){

  navLink.forEach(n => n.classList.remove('active'));
  this.classList.add('active');
  
  const navMenu = document.getElementById('nav-menu')
  navMenu.classList.remove('show')
}
navLink.forEach(n => n.addEventListener('click', linkAction));

const sr = ScrollReveal({
    origin: 'top',
    distance: '80px',
    duration: 1000,
    reset: true
});

sr.reveal('.home__title',{}); 
sr.reveal('.button',{delay: 100}); 
sr.reveal('.home__img',{delay: 150}); 
sr.reveal('.c__title',{}); 
sr.reveal('.custom-file-upload',{delay: 100}); 
var loadFile = function(event) {
	var image = document.getElementById('img_id');
	image.src = URL.createObjectURL(event.target.files[0]);
	image.style.height = "400px";
};

Overwriting static/main.js


In [8]:
%mkdir templates -p

In [9]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <link rel="stylesheet" href="{{ url_for('static',filename='style.css') }}">

        <!-- =====BOX ICONS===== -->
        <link href='https://cdn.jsdelivr.net/npm/boxicons@2.0.5/css/boxicons.min.css' rel='stylesheet'>
		<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
		<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
        <title>Finder</title>
    </head>
    <body>
        <!--===== HEADER =====-->
        <header class="l-header">
            <nav class="nav bd-grid">
                <div>
                    <a href="/" class="nav__logo">Finder</a>
                </div>

                <div class="nav__menu" id="nav-menu">
                    <ul class="nav__list">
                        <li class="nav__item"><a href="/" class="nav__link active">Home</a></li>
                        <li class="nav__item"><a href="/about" class="nav__link">About</a></li>
                    </ul>
                </div>

                <div class="nav__toggle" id="nav-toggle">
                    <i class='bx bx-menu'></i>
                </div>
            </nav>
        </header>

        <main class="l-main">
            <!--===== HOME =====-->
            <section class="home bd-grid" id="home">
                <div class="home__data">
                    <h1 class="home__title">Hi,<br>upload an <span class="home__title-color">IMAGE</span><br> and get its components</h1>
					<form action="/upload" method="post" enctype="multipart/form-data"  onsubmit="showloading()">
					
					
					<label  for="file" class="custom-file-upload">
					<input type="file"  accept="image/*" name="image" id="file" onchange="loadFile(event)" >
					Upload
					</label>

					
					
					<label class="custom-file-upload">
					<input type="submit" >
					Classify
					</label>
					</form>
                </div>
				<div></div>

                <div class="home__img" >    
                    <img src="{{ url_for('static',filename='img.svg') }}" id ="img_id" alt="">
                </div>
            </section>
			
			
            
        </main>

        <!--===== FOOTER =====-->
        <footer class="footer">
            <p class="footer__title">Finder</p>
            
            <p>&#169; 2020 copyright all right reserved</p>
        </footer>


        <!--===== SCROLL REVEAL =====-->
        <script src="https://unpkg.com/scrollreveal"></script>

        <!--===== MAIN JS =====-->
        <script src="{{ url_for('static',filename='main.js') }}"></script>
		
		
    </body>
</html>

Overwriting templates/index.html


In [10]:
%%writefile templates/predict.html
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <link rel="stylesheet" href="{{ url_for('static',filename='style.css') }}">

        <!-- =====BOX ICONS===== -->
        <link href='https://cdn.jsdelivr.net/npm/boxicons@2.0.5/css/boxicons.min.css' rel='stylesheet'>
		<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
		<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
		<script>
		function showloading() {
        image = document.getElementById('home__img');
        image.style.display = 'inline'
    }
</script>
        <title>Finder</title>
    </head>
    <body>
        <!--===== HEADER =====-->
        <header class="l-header">
            <nav class="nav bd-grid">
                <div>
                    <a href="/" class="nav__logo">Finder</a>
                </div>

                <div class="nav__menu" id="nav-menu">
                    <ul class="nav__list">
                        <li class="nav__item"><a href="/" class="nav__link active">Home</a></li>
                        <li class="nav__item"><a href="/about" class="nav__link">About</a></li>
                    </ul>
                </div>

                <div class="nav__toggle" id="nav-toggle">
                    <i class='bx bx-menu'></i>
                </div>
            </nav>
        </header>

        <main class="l-main">
            <!--===== HOME =====-->
            <section class="home bd-grid" id="home">
                <div class="home__data">
                    <h1 class="c__title"><br>the components are <br> <span class="home__title-color">{{ label }}</span><br></h1>
					<form action="/upload" method="post" enctype="multipart/form-data"  onsubmit="showloading()">
					
					
					<label  for="file" class="custom-file-upload">
					<input type="file"  accept="image/*" name="image" id="file" onchange="loadFile(event)" >
					Upload
					</label>

					
					
					<label class="custom-file-upload">
					<input type="submit" >
					Classify
					</label>
					</form>
                </div>
				<div></div>

                <div class="home__img" >    
                    <img src="{{ url_for('send_file', filename=image_file_name)}}" id ="img_id" alt="">
                </div>
				

            </section>
			
			
            
        </main>

        <!--===== FOOTER =====-->
        <footer class="footer">
            <p class="footer__title">Finder</p>
            
            <p>&#169; 2020 copyright all right reserved</p>
        </footer>


        <!--===== SCROLL REVEAL =====-->
        <script src="https://unpkg.com/scrollreveal"></script>

        <!--===== MAIN JS =====-->
        <script src="{{ url_for('static',filename='main.js') }}"></script>
		
		
    </body>
</html>

Overwriting templates/predict.html


In [11]:
%%writefile templates/about.html
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <link rel="stylesheet" href="{{ url_for('static',filename='style.css') }}">

        <!-- =====BOX ICONS===== -->
        <link href='https://cdn.jsdelivr.net/npm/boxicons@2.0.5/css/boxicons.min.css' rel='stylesheet'>
		<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
		<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
        <title>Finder</title>
    </head>
    <body>
        <!--===== HEADER =====-->
        <header class="l-header">
            <nav class="nav bd-grid">
                <div>
                    <a href="/" class="nav__logo">Finder</a>
                </div>

                <div class="nav__menu" id="nav-menu">
                    <ul class="nav__list">
                        <li class="nav__item"><a href="/" class="nav__link ">Home</a></li>
                        <li class="nav__item"><a href="/about" class="nav__link active">About</a></li>
                    </ul>
                </div>

                <div class="nav__toggle" id="nav-toggle">
                    <i class='bx bx-menu'></i>
                </div>
            </nav>
        </header>

        <main class="l-main">
        <section class="home bd-grid" id="home">
                <div class="home__data">
                    <h4>
					This web page helps detects components from images<br> based 
					on the eighteen classes presented in<br> the UC Merced Land Use dataset.
					</h4>
					<h4>The used classes :  <span class="home__title-color">['airplane', 'bare-soil','buildings',<br> 'cars',
					'chaparral', 'court', 'dock', 'field',<br> 'grass', 'mobile-home',
					'pavement', 'sand', 'sea',<br> 'ship', 'tanks', 'trees' and 'water'].</span>
					</h4>
          </div>
				<div></div>

                <div class="home__img" >    
                    <img src="{{ url_for('static',filename='img.svg') }}" id ="img_id" alt="">
                </div>
            </section>
			
			
            
        </main>

        <!--===== FOOTER =====-->
        <footer class="footer">
            <p class="footer__title">Finder</p>
            
            <p>&#169; 2020 copyright all right reserved</p>
        </footer>


        <!--===== SCROLL REVEAL =====-->
        <script src="https://unpkg.com/scrollreveal"></script>

        <!--===== MAIN JS =====-->
        <script src="{{ url_for('static',filename='main.js') }}"></script>
		
		
    </body>
</html>

Overwriting templates/about.html


In [17]:
get_ipython().system_raw(
    'pip3 install -r requirements.txt && python3 main.py > logs.txt 2>&1 &'
)
!tail logs.txt

[2020-10-03 20:02:09,044] CRITICAL in main: secret: b839bc9182f3e61e4615b73d28f54330ad561896a74571c9
 * Restarting with stat
[2020-10-03 20:02:11,313] CRITICAL in main: secret: ac5e5a1c7cf5f875c3fdae5563518f4e5a0b00827f7e02cf
 * Debugger is active!
 * Debugger PIN: 141-687-291


In [20]:
# Make sure it's running on local port
PORT = 5000

!curl http://localhost:{PORT}/hello

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>


In [21]:
!wget --quiet https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   


In [22]:
get_ipython().system_raw(
    './ngrok http {} &'
    .format(PORT)
)

In [23]:
public_url = !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

print(public_url[0])

http://427ff3957777.ngrok.io
